# **Model Server**

_______________________________________________________________________________

< INSERT INTRO >

#### **notebook take-aways**
* insert
* insert

<a id="top"></a>
#### **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#infderence)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

        !pip install -U kfserving==0.2.0 azure tensorflow keras pandas
        !pip install -U git+https://github.com/mlrun/mlrun.git#development

<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

        !pip install nuclio-jupyter

In [ ]:
# nuclio: ignore
import nuclio

Install the following packages available to the function:

In [2]:
%%nuclio cmd -c
pip install kfserving==0.2.0 --upgrade azure numpy tensorflow keras joblib

<a id="server"></a>
### _**inference server**_

In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [4]:
import kfserving
import os
import numpy as np
import joblib
from typing import List
from tensorflow.keras.models import Sequential

In [6]:
CLASSIFIER_FILE = "classifier.model.pickle"

class TFKerasClasssifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model_dir: str,
                 classifier: Sequential = None):
        """TFKerasClassifier
        
        KubeFlow serving model wrapper.
        
        :param name:            model name
        :param param model_dir: path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if classifier:
            self.ready = True

    def load(self, is_local=False):
        """Load model from KubeFlow storage.
        
        :param is_local:    (False) retrieve model from local path as
                            an alternative to kfserving.Storage, used
                            or debugging only.
        """
        if is_local:
            model_file = f"/User/projects/paysim/{CLASSIFIER_FILE}"
        else:
            model_file = os.path.join(
                kfserving.Storage.download(self.model_dir), CLASSIFIER_FILE)

        self.classifier = joblib.load(model_file)

    def predict(self, body: List) -> List:
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            feats = np.asarray(body)
            result: np.ndarray = self.classifier.predict(feats)
            return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [7]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [8]:
from mlrun import new_model_server

In [9]:
fn = new_model_server("tfkeras-server", 
                      models={"tfkeras_joblib": "/User/projects/paysim"}, 
                      model_class="Sequential")
fn.with_v3io("User","~/")

In [10]:
#fn.verbose=True
addr = fn.deploy(project="refactor-demos")

[mlrun] 2019-12-22 10:21:56,502 deploy started
[nuclio.deploy] 2019-12-22 10:21:57,585 (info) Building processor image


INFO:nuclio.deploy:(info) Building processor image


[nuclio.deploy] 2019-12-22 10:24:30,232 (info) Build complete


INFO:nuclio.deploy:(info) Build complete


[nuclio.deploy] 2019-12-22 10:25:32,874 (warn) Create function failed, setting function status


INFO:nuclio.deploy:(warn) Create function failed, setting function status


[nuclio.deploy] 2019-12-22 10:25:32,876 
Error - NuclioFunction in error state (
Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:122

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:122
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:197

Call stack:
NuclioFunction in error state (
Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:122

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:122
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:197
Failed to wait for function readiness.

Pod logs:

* tfkeras-server-5884c845f5-ptvwh
{"level":"debug","time":"2019-12-22T10:25:14.947Z","name":"processor","message":"Read configuration","more":"config=&{Config:{Meta:{Name:tfkeras-server Namespace:default-tenant Labels:map[] Annotations:map[nuclio.io/generated_by:function generated at 

INFO:nuclio.deploy:
Error - NuclioFunction in error state (
Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:122

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:122
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:197

Call stack:
NuclioFunction in error state (
Error - context deadline exceeded
    .../platform/kube/controller/nucliofunction.go:122

Call stack:
Failed to wait for function resources to be available
    .../platform/kube/controller/nucliofunction.go:122
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:197
Failed to wait for function readiness.

Pod logs:

* tfkeras-server-5884c845f5-ptvwh
{"level":"debug","time":"2019-12-22T10:25:14.947Z","name":"processor","message":"Read configuration","more":"config=&{Config:{Meta:{Name:tfkeras-server Namespace:default-tenant Labels:map[] Annotations:map[nuclio.io/generated_by:function generated at 22-12-2019 by admin f

DeployError: cannot deploy 

<a d="test"></a>
_______________________________________________________________________________

## **test deployment endpoint**

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd

import requests

In [ ]:
# Grab some data from the test set
xtest = pq.read_table("/User/projects/paysim/data/xtest.parquet").to_pandas()
ytest = pq.read_table("/User/projects/paysim/data/ytest.parquet").to_pandas()

In [ ]:
# Seldon protocol event
event_seldon = {
    "data" : {
        "ndarray": features.values.tolist()
    }
}
csel = str(event_seldon).replace("\"", "\"").replace("\n", "").replace(" ", "")

In [ ]:
resp = requests.put(addr + "/predict/tfkeras_joblib", data=csel)
print(resp.content)

<a id="inference-test"></a>
_______________________________________________________________________________

## **test saved model object**

Here we run a test outside of mlrun and the context: grab the estimated model file created on the **[kubeflow pipeline](kubeflow-pipeline.ipynb)**, load, and run a test matrix through it.

In [ ]:
import joblib
model_file = "/User/projects/paysim/classsifier.model.pickle"
lgbm_model = joblib.load(open(model_file, "rb"))

#### _**test one row**_

In [ ]:
testvals = [1.86888885,1.57777536,-0.39419246,2.05800271,-0.75799805,1.57381201,1.4090718,1.26302791,1.08653808,1.22390795,0.46826237,0.78775787,0.,0.84368491,0.33878541,-0.4717221,2.54822445,0.38823441,-0.57677436,1.70674813,0.,0.8656919,0.87541604,0.98195601,2.04881859,1.63834357,1.42886198,1.33937621]
testvals = np.asarray(testvals).reshape(-1,1).T

In [ ]:
lgbm_model.predict(testvals)

#### _**test matrix**_

In [ ]:
lgbm_model.predict(features)